In [1]:
import numpy as np # 导入NumPy库，用于进行科学计算和数组操作
import pandas as pd # 导入Pandas库，用于数据处理和分析，主要使用DataFrame
import scipy.stats as stats # 导入SciPy的统计模块，用于计算偏度、峰度等
import scipy # 导入SciPy库
# import pymysql  # 导入数据库模块 (此处被注释掉，未使用)

from datetime import datetime # 从datetime模块导入datetime类，用于处理时间格式
import statsmodels.formula.api as smf  # 导入statsmodels的公式API，用于执行OLS回归分析

# import pyreadr # read RDS file (此处被注释掉，未使用)

from matplotlib import style # 导入matplotlib的样式模块
import matplotlib.pyplot as plt  # 导入matplotlib的绘图模块
import matplotlib.dates as mdates # 导入matplotlib的日期处理模块，用于x轴日期格式化

from matplotlib.font_manager import FontProperties # 用于处理绘图中的字体（如中文显示）
from pylab import mpl # 导入pylab的mpl模块
#mpl.rcParams['font.sans-serif'] = ['SimHei'] # 设置绘图的中文字体为黑体 (注释掉)
#plt.rcParams['font.family'] = 'Times New Roman' # 设置英文字体 (注释掉)

#输出矢量图 渲染矢量图，使jupyter notebook中的图片更清晰
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

from IPython.core.interactiveshell import InteractiveShell # 导入IPython交互设置模块
#设置显示每一个运行结果，而不仅仅是最后一行
InteractiveShell.ast_node_interactivity = 'all'

#设置行不限制数量 (注释掉)
#pd.set_option('display.max_rows',None)

#设置列不限制数量，打印dataframe时显示所有列
pd.set_option('display.max_columns', None)

In [6]:
# 读取csv文件，加载个股000001的日度交易数据
data = pd.read_csv('E:\BaiduNetdiskDownload/000001.csv')
# 将'Day'列转换为datetime时间格式
data['Day'] = pd.to_datetime(data['Day'],format='%Y/%m/%d')
# 将'Day'列设置为索引(Index)，方便按时间切片和重采样
data.set_index('Day', inplace = True)
# 按照时间(Index)升序排列数据
data.sort_values(by = ['Day'],axis=0, ascending=True)

# --- Cell 3 ---
# 选取1995年1月到2024年9月的数据并拷贝一份，避免并在原数据上修改
data_new = data['1995-01':'2024-09'].copy()
# 将'Close'（收盘价）列转换为数字类型，防止数据中包含字符串
data_new['Close'] = pd.to_numeric(data_new['Close'])
# 将'Preclose'（前收盘价）列转换为数字类型
data_new['Preclose'] = pd.to_numeric(data_new['Preclose'])
# 计算原始日收益率：(今收/前收) - 1
data_new['Raw_return'] = data_new['Close'] / data_new['Preclose'] - 1
# 显示处理后的数据
data_new

,Preclose,Open,Highest,Lowest,Close
Day,,,,,
1990-12-19,,96.050,99.980,95.790,99.980
1990-12-20,99.98,104.300,104.390,99.980,104.390
1990-12-21,104.39,109.070,109.130,103.730,109.130
1990-12-24,109.13,113.570,114.550,109.130,114.550
1990-12-25,114.55,120.090,120.250,114.550,120.250
...,...,...,...,...,...
2025-08-25,3825.759,3848.163,3883.562,3839.972,3883.562
2025-08-26,3883.562,3871.471,3888.599,3859.758,3868.382
2025-08-27,3868.382,3869.612,3887.198,3800.350,3800.350


,Preclose,Open,Highest,Lowest,Close,Raw_return
Day,,,,,,
1995-01-03,647.870,637.720,647.710,630.530,639.880,-0.012333
1995-01-04,639.880,641.900,655.510,638.860,653.810,0.021770
1995-01-05,653.810,655.380,657.520,645.810,646.890,-0.010584
1995-01-06,646.890,642.750,643.890,636.330,640.760,-0.009476
1995-01-09,640.760,637.520,637.550,625.040,626.000,-0.023035
...,...,...,...,...,...,...
2024-09-24,2748.918,2770.754,2863.152,2761.372,2863.126,0.041547
2024-09-25,2863.126,2901.419,2952.451,2889.048,2896.306,0.011589
2024-09-26,2896.306,2893.745,3000.953,2889.014,3000.953,0.036131


In [ ]:
# 将数据按月度末('ME')重采样
# 对'Raw_return'列应用函数：计算(1+r)的累乘再减1，得到月度复利收益率
# .to_frame()将结果从Series转换回DataFrame
Month_data = data_new.resample('ME')['Raw_return'].apply(lambda x: (1+x).prod() - 1).to_frame()
# 重置索引，将Index变回普通列
Month_data.reset_index(inplace = True)
# 将列名'Day'重命名为'month'
Month_data.rename(columns = {'Day':'month'}, inplace = True)
# 重新将'month'设置为索引
Month_data.set_index('month', inplace = True)
# 显示月度数据
Month_data

# 逻辑同上，'QE'代表Quarter End（季度末）
Quarter_data = data_new.resample('QE')['Raw_return'].apply(lambda x: (1+x).prod() - 1).to_frame()
Quarter_data.reset_index(inplace = True)
Quarter_data.rename(columns = {'Day':'Q'}, inplace = True) # 重命名为Q
Quarter_data.set_index('Q', inplace = True)
Quarter_data

# 逻辑同上，'YE'代表Year End（年末）
Year_data = data_new.resample('YE')['Raw_return'].apply(lambda x: (1+x).prod() - 1).to_frame()
Year_data.reset_index(inplace = True)
Year_data.rename(columns = {'Day':'Year'}, inplace = True)
Year_data.set_index('Year', inplace = True)
Year_data

,Raw_return
month,
1995-01-31,-0.131631
1995-02-28,-0.023694
1995-03-31,0.177803
1995-04-30,-0.103552
1995-05-31,0.207922
...,...
2024-05-31,-0.005801
2024-06-30,-0.038684
2024-07-31,-0.009656


,Raw_return
Q,
1995-03-31,-0.001466
1995-06-30,-0.025258
1995-09-30,0.145660
1995-12-31,-0.231358
1996-03-31,0.001981
...,...
2023-09-30,-0.028603
2023-12-31,-0.043575
2024-03-31,0.022263


,Raw_return
Year,
1995-12-31,-0.142899
1996-12-31,0.651425
1997-12-31,0.302153
1998-12-31,-0.039695
1999-12-31,0.191750
2000-12-31,0.517277
2001-12-31,-0.206180
2002-12-31,-0.175167
2003-12-31,0.102670


In [ ]:
# 读取通胀数据
inflation = pd.read_csv('datasets/inflation.csv')
# 将'month'列转换为datetime格式
inflation['month'] = pd.to_datetime(inflation['month'],format='%Y/%m/%d')
# 设置'month'为索引
inflation.set_index('month',inplace=True)
# 按时间排序
inflation.sort_values(by=['month'],axis=0,ascending=True)

In [ ]:
# 将数据按月重采样
# 计算月内所有日收益率的平方和，作为月度市场波动率(Variance)的代理变量
market_variance = data_new.resample('ME').apply({
    'Raw_return':
    lambda x: sum(x**2)
})
# 重置索引
market_variance.reset_index(inplace=True)
# 重命名列：Day -> month, Raw_return -> RV (Realized Volatility)
market_variance.rename(columns={'Day':'month','Raw_return':'RV'},inplace=True)
# 设置索引
market_variance.set_index('month',inplace=True)
# 显示数据
market_variance

In [ ]:
# 读取包含多只股票月度数据的csv文件
cross = pd.read_csv('datasets/cross_section2023.csv')
from pandas.tseries.offsets import MonthEnd # 导入MonthEnd偏移量
# 将'month'转换为datetime格式，并加上MonthEnd(1)确保日期是每个月的最后一天（与上面数据对齐）
cross['month'] = pd.to_datetime(cross['month'], format='%Y-%m-%d') + MonthEnd(1)
# 将换手率(to_v)和流通市值(floatingvalue)转换为数字类型
cross['to_v'] = pd.to_numeric(cross['to_v'])
cross['floatingvalue'] = pd.to_numeric(cross['floatingvalue'])
# 删除这两列中有缺失值的行
cross = cross.dropna(subset=['to_v','floatingvalue'])
# 删除所有包含缺失值的行
cross = cross.dropna()
# 补齐股票代码：将代码转换为字符串，如果不满6位，在左侧补0
cross['Stkcd'] = cross['Stkcd'].apply(lambda x: '{:0>6}'.format(x)) 
# 创建一个辅助列'w'，全为1（Cell 10中用于计算等权平均）
cross['w'] = 1
cross
# 按月分组，计算换手率的加权平均（这里权重w是1，所以是等权平均）
turnover = pd.DataFrame(cross.groupby(['month']).apply(
    lambda x:
    np.average(x['to_v'],weights=x['w']) 
, include_groups=False)) # include_groups=False 防止报错(新版Pandas特性)
# 重命名列为'to'
turnover = turnover.rename(columns={0:'to'})
turnover
# 按月分组，计算换手率的加权平均，权重是'floatingvalue'（流通市值）
# 这通常是更准确的市场换手率指标
turnover = pd.DataFrame(cross.groupby(['month']).apply(
    lambda x:
    np.average(x['to_v'],weights=x['floatingvalue']) 
, include_groups=False))
turnover = turnover.rename(columns={0:'to'})
turnover

In [ ]:
# 将月度回报率和波动率合并
reg_data = pd.merge(Month_data,market_variance,on = 'month')
# 合并通胀数据
reg_data = pd.merge(reg_data,inflation,on = 'month')
# 合并换手率数据
reg_data = pd.merge(reg_data,turnover,on='month')
# 截取2000年及以后的数据用于回归分析
reg_data = reg_data['2000':]
reg_data
# 查看换手率('to')的统计量：计数、均值、标准差、最小/最大值等，保留5位小数
reg_data['to'].describe().round(5)

# --- Cell 14: 偏度与峰度 ---
# 计算换手率的偏度 (Skewness)
reg_data['to'].skew()
# 计算换手率的峰度 (Kurtosis)
reg_data['to'].kurt()

In [ ]:
fig = plt.figure(figsize=(10, 5)) # 图片比例
ax = fig.add_subplot(1, 1, 1)  # 定义ax
ax.plot(
    'RV',  # 要画图的变量名
    '.-r',  # 线的类型
    linewidth = 1,  # 线的粗细
    data = reg_data['2000-01-01':'2023-12-31'])  # 画图的数据
plt.title("China's Stock Market Variance") # 画图的标题
plt.xlabel('Month') # 画图的x轴名称
plt.ylabel('Variance') # 画图的y轴名称

# 设置x轴的日期显示格式
data_format = mdates.DateFormatter('%Y')
ax.xaxis.set_major_formatter(data_format)
ax.xaxis.set_major_locator(mdates.YearLocator())

# 转置x轴的日期显示格式
plt.xticks(rotation = 90)

fig.savefig('Variance.pdf', bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 5)) # 图片比例
ax = fig.add_subplot(1, 1, 1)  # 定义ax
ax.plot(
    'to',  # 要画图的变量名
    '.-r',  # 线的类型
    linewidth = 1,  # 线的粗细
    data = reg_data['2000-01-01':'2022-12-31'])  # 画图的数据
plt.title("China's Stock Market Turnover") # 画图的标题
plt.xlabel('Month') # 画图的x轴名称
plt.ylabel('Turnover') # 画图的y轴名称

# 设置x轴的日期显示格式
data_format = mdates.DateFormatter('%Y')
ax.xaxis.set_major_formatter(data_format)
ax.xaxis.set_major_locator(mdates.YearLocator())

# 转置x轴的日期显示格式
plt.xticks(rotation = 90)

fig.savefig('Turnover.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Change the figure style
# list the figure style
print(plt.style.available)
#plt.style.use('ggplot')
# 使用默认的画图风格
plt.style.use('default')

fig = plt.figure(figsize=(10, 5))
ax1 = fig.add_subplot(1, 1, 1)  #(x, x, x)这里前两个表示几*几的网格，最后一个表示第几子图

ax1.plot(reg_data['RV'],
         color='red',
         marker='.',
         linestyle='-',
         linewidth=1,
         markersize=6,
         alpha=0.4,
         label='Market Vaiance')
ax1.set_xlabel('Month')  # 设置横坐标标签
ax1.set_ylabel('Return')  # 设置左边纵坐标标签
#ax1.legend(loc=2)  # 设置图例在左上方
ax1.set_title("China Stock Market Turnover and Variance")  # 给整张图命名

# 设置x轴的日期显示格式
data_format = mdates.DateFormatter('%Y')
ax1.xaxis.set_major_formatter(data_format)
ax1.xaxis.set_major_locator(mdates.YearLocator())
# 转置x轴的日期显示格式
plt.xticks(rotation = 90)

ax2 = ax1.twinx()  #twinx()函数表示共享x轴
ax2.plot(reg_data['to'],
         color='blue',
         marker='o',
         linestyle='-',
         linewidth=1,
         markersize=2,
         alpha=0.7,
         label='Turnover')
ax2.set_ylabel('Turnover')  # 设置右边纵坐标标签
#ax2.legend(loc=1)  # 设置图例在右上方

# change the legend into one box
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper left')

fig = plt.gcf()
fig.savefig('mvto.pdf', bbox_inches='tight')
plt.show()

In [ ]:
#  AR(1)模型，用滞后一期的RV预测当期RV 
# 创建滞后一期的波动率 lRV
reg_data['lRV'] = reg_data['RV'].shift(1)
# 建立OLS回归模型：RV = alpha + beta * lRV
# cov_type='HAC' 表示使用异方差自相关一致的协方差矩阵 (Newey-West调整)
# maxlags=6 表示考虑6阶滞后
model_fore_mv = smf.ols('RV ~ lRV',
                 data=reg_data['2000-01':'2023-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 6})
# 打印回归结果
print(model_fore_mv.summary())

#  用滞后一期的换手率预测波动率
# 创建滞后一期的换手率 lto
reg_data['lto'] = reg_data['to'].shift(1)
# 回归模型：RV = alpha + beta * lto
model_fore_mv = smf.ols('RV ~ lto',
                 data=reg_data['2000-01':'2023-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 6})
print(model_fore_mv.summary())

# --- Cell 35: 联合预测模型 ---
# 回归模型：RV = alpha + beta1 * lto + beta2 * lRV
# 同时使用滞后换手率和滞后波动率进行预测
reg_data['lRV'] = reg_data['RV'].shift(1) # 再次确认lRV存在
model_fore_mv = smf.ols('RV ~ lto + lRV',
                 data=reg_data['2000-01':'2023-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 6})
print(model_fore_mv.summary())

In [ ]:
# 模型（联合模型）的拟合值（预测的波动率）保存到'fitted_mv'列
reg_data['fitted_mv'] = model_fore_mv.fittedvalues
reg_data
# 回归模型：Return = alpha + beta * Predicted_Variance
# 测试风险-收益权衡
model_fore_ret = smf.ols('Raw_return ~ fitted_mv',
                 data=reg_data['2000-01':'2023-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 6})
print(model_fore_ret.summary())

In [ ]:
fig = plt.figure(figsize=(10, 5)) # 创建画布
ax1 = fig.add_subplot(1, 1, 1)  # 添加子图

# 绘制第一条线：实际市场波动率 (RV)
ax1.plot(reg_data['RV'],
         color='blue', # 蓝色
         marker='.', # 点标记
         linestyle='-', # 实线
         linewidth=1, # 线宽
         label='Market Variance') # 图例标签

plt.xlabel('month')  # x轴标签
plt.ylabel('Variance')  # y轴标签
plt.title("Variance and Turnover: Monthly 1995-2023")  # 标题

# 绘制第二条线：拟合/预测的波动率 (fitted_mv)
ax1.plot(reg_data['fitted_mv'],
         color='red', # 红色
         marker='o', # 圆圈标记
         linestyle='-', # 实线
         linewidth=1, # 线宽
         markersize=2, # 标记大小
         alpha=0.7, # 透明度0.7
         label='fitted_mv') # 图例标签

# 设置x轴日期格式
data_format = mdates.DateFormatter('%Y')
ax1.xaxis.set_major_formatter(data_format)
ax1.xaxis.set_major_locator(mdates.YearLocator(1)) # 刻度间隔为1年

# 旋转x轴标签
plt.xticks(rotation = 90)

# 显示图例，位置在右上角
plt.legend(loc='upper right')

# 保存并显示
fig.savefig('MV_fitted.pdf', bbox_inches='tight')
plt.show();

In [ ]:
# 确保lto列存在
reg_data['lto'] = reg_data['to'].shift(1)
# 回归模型：Return = alpha + beta * Lagged_Turnover
model_to = smf.ols('Raw_return ~ lto',
                 data=reg_data['2000-01':'2023-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 6})
print(model_to.summary())

In [ ]:
# 查看当前数据
reg_data

# Raw_return: 复利累乘
# to (换手率): 季度内求和 (换手率是流量概念)
# cpi: 季度内求和
Qreg_data = reg_data.resample('QE').apply({
    'Raw_return':
    lambda x: np.prod(1+x) - 1,
    'to':
    lambda x: sum(x),
    'cpi':
    lambda x: sum(x)
})
Qreg_data

# 季度回归
# 创建季度滞后项
Qreg_data['lto'] = Qreg_data['to'].shift(1)
Qreg_data['lcpi'] = Qreg_data['cpi'].shift(1)
# 回归模型：Quarterly_Return ~ Lag_Turnover + Lag_CPI
model_to = smf.ols('Raw_return ~ lto + lcpi',
                 data=Qreg_data['2000-01':'2023-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 2}) # 季度数据lag设为2
print(model_to.summary())

# 7: 汇总回归结果表
from statsmodels.iolib.summary2 import summary_col # 导入用于生成汇总表的库

# 定义三个模型：单变量换手率、单变量CPI、多变量
model_to = smf.ols('Raw_return ~ lto', data=Qreg_data['2000-01':'2023-12']).fit(cov_type='HAC', cov_kwds={'maxlags': 2})
model_cpi = smf.ols('Raw_return ~ lcpi', data=Qreg_data['2000-01':'2023-12']).fit(cov_type='HAC', cov_kwds={'maxlags': 2})
model_cpito = smf.ols('Raw_return ~ lto + lcpi', data=Qreg_data['2000-01':'2023-12']).fit(cov_type='HAC', cov_kwds={'maxlags': 2})

# 生成汇总表格
results_table = summary_col(results=[model_to, model_cpi, model_cpito],
                            float_format='%0.3f', # 保留3位小数
                            stars=True, # 显示显著性星号
                            model_names=['Quarter Turnover', 'Quarter Inflation', 'Quarter CPI & Inflation'],
                            info_dict={'No. observations': lambda x: f"{int(x.nobs):d}"},
                            regressor_order=['Intercept', 'lto','lcpi']) # 变量显示顺序

results_table.add_title('Table - OLS Regressions: Forecast Quarterly Stock Market Return')
print(results_table)

In [ ]:
# 读取全市场回报率数据
Marketret_allstock = pd.read_csv('datasets/Marketret_allstock.csv',encoding='gbk')
# 生成月度时间序列
Marketret_allstock['month'] = pd.date_range(start='1991', end='2023', freq='ME')
# 保留关键列
Marketret_allstock = Marketret_allstock[['month','MarketR','rfmonth','ret']]
Marketret_allstock

# 合并数据 -
# 重建基础数据集 reg_data (Month_data, market_variance, inflation, turnover)
reg_data = pd.merge(Month_data,market_variance,on = 'month')
reg_data = pd.merge(reg_data,inflation,on = 'month')
reg_data = pd.merge(reg_data,turnover,on='month')
reg_data = reg_data[reg_data.index >= '2000-01']
# 将全市场回报率合并进来 (左连接)
reg_data = pd.merge(reg_data,Marketret_allstock,on='month',how='left')
# 创建滞后项
reg_data['lto'] = reg_data['to'].shift(1) # 滞后1期换手率
reg_data['lcpi'] = reg_data['cpi'].shift(2) # 注意：这里CPI滞后了2期 (可能是因为CPI发布有延迟)
# 重新设置索引
reg_data.set_index('month',inplace=True)
reg_data

#  全市场回归汇总表
# 模型1：ret ~ lto
# 模型2：ret ~ lcpi
# 模型3：ret ~ lto + lcpi
# --- Cell 42 ---
from statsmodels.iolib.summary2 import summary_col # 导入用于生成汇总表的工具

# 定义信息字典，用于在表格底部显示观测值数量
info_dict = {'No. observations': lambda x: f"{int(x.nobs):d}"}

# 模型1：单变量回归，用滞后换手率预测全市场回报率 (ret)
model_to = smf.ols('ret ~ lto',
                 data=reg_data['2000-01':'2022-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 2}) # HAC标准误，滞后2阶

# 模型2：单变量回归，用滞后CPI预测全市场回报率
model_cpi = smf.ols('ret ~ lcpi',
                 data=reg_data['2000-01':'2022-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 2})

# 模型3：多变量回归，同时使用滞后换手率和滞后CPI预测全市场回报率
model_cpito = smf.ols('ret ~ lto + lcpi',
                 data=reg_data['2000-01':'2022-12']).fit(
                     cov_type='HAC', cov_kwds={'maxlags': 2})

# 生成汇总表格
results_table = summary_col(results=[model_to, model_cpi, model_cpito], # 传入三个模型结果
                            float_format='%0.3f', # 数据保留3位小数
                            stars=True, # 显示显著性星号 (* p<.1, ** p<.05, ***p<.01)
                            model_names=['Turnover', 'Inflation', 'CPI & Inflation'], # 定义模型列名
                            info_dict=info_dict, # 添加底部信息
                            regressor_order=['Intercept', 'lto','lcpi']) # 指定自变量显示顺序

# 添加表格标题
results_table.add_title(
    'Table - OLS Regressions: Forecast Monthly Stock Market Return')

print(results_table)